In [1]:
import time
a = time.time()
import os
# Overwrite Pyspark driver
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"

In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline
from pyspark.sql import SparkSession
from langdetect import detect
from pyspark.sql.functions import when, lit, col, udf, concat_ws
import pyspark as ps
from pyspark.sql import types as t
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

2022-05-01 20:24:06.616767: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 20:24:06.616857: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
spark = SparkSession.builder \
    .appName("Redit Summarization")\
    .master("yarn")\
    .config("spark.driver.memory","16g")\
    .config("spark.executor.memoryOverhead","2048")\
    .config("spark.executor.memory","4g")\
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.yarn.dist.archives","pyspark_venv.tar.gz#environment")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.3,org.apache.hadoop:hadoop-aws:2.7.3")\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-102b0e15-4275-4032-a609-6a60922205a5;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.3 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastu

	0 artifacts copied, 84 already retrieved (0kB/37ms)
2022-05-01 20:24:32,701 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-05-01 20:24:40,130 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-05-01 20:24:41,946 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2022-05-01 20:25:01,703 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-3.4.3.jar added multiple times to distributed cache.
2022-05-01 20:25:01,703 WARN yarn.Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.apache.hadoop_hadoop-aws-2.7.3.jar added multiple times to distributed cache.
2022-05-01 20:25:01,703 WARN yarn.Client: Same 

In [4]:
# Read data
df = spark.read.format("csv").option("header","true").load("hdfs://namenode:9000/popular_comments.csv")

In [5]:
df = df.na.drop()

In [6]:
document_assembler = DocumentAssembler()\
.setInputCol("comment")\
.setOutputCol("documents")


In [7]:
t5 = T5Transformer() \
  .load("hdfs://namenode:9000/dis_materials/t5_small_en_2.7.1_2.4_1610133219885") \
  .setTask("summarize:")\
  .setMaxOutputLength(100)\
  .setInputCols(["documents"]) \
  .setOutputCol("summaries")

2022-05-01 20:25:46.338857: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-01 20:25:46.488505: I external/org_tensorflow/tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2399995000 Hz
2022-05-01 20:25:47.799669: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 65798144 exceeds 10% of free system memory.
2022-05-01 20:25:48.113246: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 65798144 exceeds 10% of free system memory.
2022-05-01 20:25:48.224970: W external/org_tensorflow/tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 65798144 exceeds 10% of free system memory.
20

In [8]:
summarizer_pp = Pipeline(stages=[
    document_assembler, t5
])

In [9]:
empty_df = spark.createDataFrame([['']]).toDF('comment')
pipeline_model = summarizer_pp.fit(empty_df)
sum_lmodel = LightPipeline(pipeline_model)

In [10]:
empty_df.take(2)

[Row(comment='')]

In [19]:
new_df = sum_lmodel.transform(df)

In [12]:
# df = new_df.persist(ps.StorageLevel.DISK_ONLY)

In [20]:
new_df = new_df.withColumn("results",col("summaries.result"))

In [21]:
new_df = new_df.withColumn("result",concat_ws(",",col("results")))

In [23]:
new_df = new_df.select("subreddit","max_ups","result")

In [24]:
new_df.columns

['subreddit', 'max_ups', 'result']

In [16]:
b = time.time()
b-a

126.6496217250824

In [25]:
new_df.write.option("header","true").csv("hdfs://namenode:9000/summarized_data.csv")

----------------------------------------                            (0 + 2) / 2]
Exception happened during processing of request from ('127.0.0.1', 35832)
Traceback (most recent call last):
  File "/usr/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/accumulators.py", line 239, in a

Py4JError: An error occurred while calling o326.csv